## 뉴욕시의 Yellow Taxi Trip데이터셋

In [56]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('yellowtaxi_trip_count').getOrCreate()

In [57]:
import os

trip_files = '/trips/*'
zone_files = 'taxi+_zone_lookup.csv'
directory = os.path.join(os.getcwd(), 'data')

In [58]:
trips_df = spark.read.csv(f'file:///{directory}/{trip_files}',inferSchema=True, header=True)

In [59]:
zone_df = spark.read.csv(f'file:///{directory}/{zone_files}',inferSchema=True, header=True)

In [60]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [61]:
trips_df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2021-03-01 00:22:02|  2021-03-01 00:23:22|              1|          0.0|         1|                 N|         264|         264|           2|        3.0|  0.5|    0.5|       0.0|         0.0|                  0.3

In [62]:
trips_df.createOrReplaceTempView('trips')
zone_df.createOrReplaceTempView('zone')

In [63]:
query = '''
select
t.VendorID,
to_date(t.tpep_pickup_datetime) as pickup_date,
to_date(t.tpep_dropoff_datetime) as dropoff_date,
hour(t.tpep_dropoff_datetime) as pickup_time,
hour(t.tpep_dropoff_datetime) as dropoff_time,
t.passenger_count,
t.trip_distance,
t.tip_amount,
t.total_amount,
t.payment_type,
pz.Zone as pickup_zone,
dz.Zone as dropoff_zone
from trips as t
left join zone as pz
on t.PULocationID = pz.LocationID
left join zone as dz
on t.DOLocationID = dz.LocationID
'''

df = spark.sql(query)
df.show(5)

+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------------+--------------+
|VendorID|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|tip_amount|total_amount|payment_type|      pickup_zone|  dropoff_zone|
+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------------+--------------+
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         4.3|           2|               NV|            NV|
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         3.8|           2|   Manhattanville|Manhattanville|
|       2| 2021-03-01|  2021-03-01|          0|           0|              1|          0.0|       0.0|         4.8|           2|   Manhattanville|Manhattanville|
|       1| 2021-03-01|  2021-03-01

In [64]:
df.count()

15000700

## 데이터 전처리

## 열
- **VendorID**: 택시 운행을 제공한 공급자의 식별자.
- **pickup_date**: 택시 승차 일자 (tpep_pickup_datetime에서 변환된 날짜).
- **dropoff_date**: 택시 하차 일자 (tpep_dropoff_datetime에서 변환된 날짜).
- **pickup_time**: 승차 시간 (tpep_pickup_datetime에서 시간 추출).
- **dropoff_time**: 하차 시간 (tpep_dropoff_datetime에서 시간 추출).
- **passenger_count**: 탑승한 승객 수.
- **trip_distance**: 여행의 총 거리 (마일 단위).
- **tip_amount**: 승객이 지불한 팁 금액.
- **total_amount**: 여행의 총 요금 (팁 포함).
- **payment_type**: 결제 방법 (예: 카드, 현금 등).
- **pickup_zone**: 택시 승차 지역 (zone 테이블에서 가져온 값).
- **dropoff_zone**: 택시 하차 지역 (zone 테이블에서 가져온 값).

## 중복 데이터 제거

In [9]:
df.count() - df.distinct().count()

16074

In [10]:
df = df.dropDuplicates()

## 결측치 & 이상치 확인
- VendorID, passenger_count, payment_type 열에 각각 834,028개의 null 값이 존재
- 결측 비율은 약 5.56%
- VendorID : 최빈값 대체
- passenger_count : 0명은 이상치로 간주하여 제거, 중앙값 대체
- payment_type : 최빈값 대체
- tip_amount, total_amount 음수인 경우 제거

In [8]:
# 결측치 확인
import pyspark.sql.functions as F

null_counts = df.select(*[F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)
                         for c in df.columns])
null_counts.show()

+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------+------------+
|VendorID|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|tip_amount|total_amount|payment_type|pickup_zone|dropoff_zone|
+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------+------------+
|  834028|          0|           0|          0|           0|         834028|            0|         0|           0|      834028|          0|           0|
+--------+-----------+------------+-----------+------------+---------------+-------------+----------+------------+------------+-----------+------------+



In [9]:
# 결측치 비율
total_rows = df.count()

null_ratios = null_counts.select(
            *[
        (F.col(c) / total_rows).alias(f"{c}_null_ratio") 
        for c in null_counts.columns
])

null_ratios.show()

+-------------------+----------------------+-----------------------+----------------------+-----------------------+--------------------------+------------------------+---------------------+-----------------------+-----------------------+----------------------+-----------------------+
|VendorID_null_ratio|pickup_date_null_ratio|dropoff_date_null_ratio|pickup_time_null_ratio|dropoff_time_null_ratio|passenger_count_null_ratio|trip_distance_null_ratio|tip_amount_null_ratio|total_amount_null_ratio|payment_type_null_ratio|pickup_zone_null_ratio|dropoff_zone_null_ratio|
+-------------------+----------------------+-----------------------+----------------------+-----------------------+--------------------------+------------------------+---------------------+-----------------------+-----------------------+----------------------+-----------------------+
|0.05559927203397175|                   0.0|                    0.0|                   0.0|                    0.0|       0.05559927203397175|   

In [11]:
df.select('VendorID','passenger_count','payment_type').describe().show()

+-------+------------------+------------------+------------------+
|summary|          VendorID|   passenger_count|      payment_type|
+-------+------------------+------------------+------------------+
|  count|          14150660|          14150660|          14150660|
|   mean|1.6875516760348988|1.4257929312131024|1.2573771117389578|
| stddev|0.4634915144829814|1.0446880103219505|0.4772950890499679|
|    min|                 1|                 0|                 1|
|    max|                 2|                 9|                 5|
+-------+------------------+------------------+------------------+



In [10]:
# VendorID
df.groupBy('VendorID').count().show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|    null| 834028|
|       1|4431189|
|       2|9735483|
+--------+-------+



In [11]:
df = df.fillna({'VendorID': 2})

In [11]:
# passenger_count
df.groupBy('passenger_count').count().show()

+---------------+--------+
|passenger_count|   count|
+---------------+--------+
|           null|  834028|
|              1|10472413|
|              6|  238647|
|              3|  535718|
|              5|  357257|
|              9|      18|
|              4|  202839|
|              8|      22|
|              7|      33|
|              2| 2041604|
|              0|  318121|
+---------------+--------+



In [12]:
median_passenger_count = df.approxQuantile('passenger_count', [0.5], 0.01)[0]  
df = df.fillna({'passenger_count': median_passenger_count})

In [13]:
df = df.filter(df['passenger_count'] != 0)

In [12]:
# payment_type
df.groupBy('payment_type').count().show()

+------------+--------+
|payment_type|   count|
+------------+--------+
|        null|  834028|
|           1|10716903|
|           3|   81434|
|           5|       1|
|           4|   59664|
|           2| 3308670|
+------------+--------+



In [14]:
df = df.fillna({'payment_type': 1})

In [17]:
df.select('tip_amount').describe().show()

+-------+-----------------+
|summary|       tip_amount|
+-------+-----------------+
|  count|         15000700|
|   mean|2.146797558780939|
| stddev|2.610914434555077|
|    min|          -333.32|
|    max|          1140.44|
+-------+-----------------+



In [18]:
df.filter(F.col('tip_amount') < 0).count()

1011

In [15]:
# 팁 금액이 음수인 경우 제거
import pyspark.sql.functions as F

df = df.filter(F.col('tip_amount') >= 0)
df.count()

14666030

In [25]:
df.select(F.col('tip_amount')).describe().show()

+-------+------------------+
|summary|        tip_amount|
+-------+------------------+
|  count|          14931876|
|   mean|2.1567248181009013|
| stddev| 2.609006208413183|
|    min|               0.0|
|    max|           1140.44|
+-------+------------------+



In [20]:
df.select('total_amount').describe().show()

+-------+------------------+
|summary|      total_amount|
+-------+------------------+
|  count|          14999689|
|   mean|18.757396086862318|
| stddev| 145.7488478519927|
|    min|            -647.8|
|    max|          398469.2|
+-------+------------------+



In [21]:
df.filter(F.col('total_amount') < 0).count()

67813

In [16]:
# 총 금액이 음수이거나 0인 경우 제거
df = df.filter(F.col('total_amount') > 0)
df.count()

14594895

In [24]:
df.select(F.col('total_amount')).describe().show()

+-------+------------------+
|summary|      total_amount|
+-------+------------------+
|  count|          14931876|
|   mean|18.908677106800845|
| stddev| 146.0575764264735|
|    min|               0.0|
|    max|          398469.2|
+-------+------------------+



## 파생 변수 생성
- tip_ratio 칼럼 생성

In [17]:
# 팁의 비율(팁/총 금액) 컬럼 추가 
df = df.withColumn('tip_ratio',F.col('tip_amount') / F.col('total_amount'))
df.select(F.col('tip_ratio')).describe().show()

+-------+-------------------+
|summary|          tip_ratio|
+-------+-------------------+
|  count|           14594895|
|   mean|0.11334367404538742|
| stddev|0.08268506886335779|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [18]:
null_count_tip_ratio = df.select(
    F.sum(F.when(F.col('tip_ratio').isNull(), 1).otherwise(0)).alias('null_count')
)

null_count_tip_ratio.show()

+----------+
|null_count|
+----------+
|         0|
+----------+



## 상관계수

In [27]:
df.printSchema()

root
 |-- VendorID: integer (nullable = false)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- dropoff_time: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = false)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)
 |-- tip_ratio: double (nullable = true)



In [33]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

numeric_columns = ['tip_amount', 'total_amount', 'trip_distance', 'tip_ratio'] 
assembler = VectorAssembler(inputCols=numeric_columns, outputCol='features')

# 벡터화된 데이터를 생성
df_vec = assembler.transform(df)

# 상관행렬 계산
correlation_matrix = Correlation.corr(df_vec, 'features').head()[0]  
print(correlation_matrix)

24/12/16 11:27:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/16 11:27:48 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


DenseMatrix([[ 1.00000000e+00,  5.39075142e-02,  4.22992711e-04,
               6.34515553e-01],
             [ 5.39075142e-02,  1.00000000e+00,  1.13608863e-03,
               9.70655138e-04],
             [ 4.22992711e-04,  1.13608863e-03,  1.00000000e+00,
              -5.47965311e-03],
             [ 6.34515553e-01,  9.70655138e-04, -5.47965311e-03,
               1.00000000e+00]])


In [34]:
import pandas as pd


correlation_array = correlation_matrix.toArray()
correlation_df = pd.DataFrame(correlation_array, columns=numeric_columns, index=numeric_columns)
print(correlation_df)

               tip_amount  total_amount  trip_distance  tip_ratio
tip_amount       1.000000      0.053908       0.000423   0.634516
total_amount     0.053908      1.000000       0.001136   0.000971
trip_distance    0.000423      0.001136       1.000000  -0.005480
tip_ratio        0.634516      0.000971      -0.005480   1.000000


In [35]:
# tip_ratio 칼럼 제거
df = df.drop('tip_ratio')

## dropoff_zone, pickup_zone 변수 제거

In [39]:
df.groupBy("dropoff_zone").count().show()

+--------------------+------+
|        dropoff_zone| count|
+--------------------+------+
|           Homecrest|  4314|
|Governor's Island...|     7|
|              Corona|  6792|
|    Bensonhurst West|  4420|
|         Westerleigh|   224|
|      Newark Airport| 17319|
|Charleston/Totten...|   825|
|          Douglaston|  2035|
|          Mount Hope|  6515|
|East Concourse/Co...| 11984|
|      Pelham Parkway|  4114|
|         Marble Hill|  1285|
|           Rego Park|  7604|
|Upper East Side S...|659445|
|       Dyker Heights|  2614|
|Heartland Village...|   397|
|   Kew Gardens Hills|  5333|
|     Jackson Heights| 24488|
|             Bayside|  3538|
|      Yorkville West|348092|
+--------------------+------+
only showing top 20 rows



In [40]:
df.groupBy("pickup_zone").count().show()

+--------------------+------+
|         pickup_zone| count|
+--------------------+------+
|           Homecrest|  1922|
|Governor's Island...|     7|
|              Corona|  2280|
|    Bensonhurst West|  2064|
|         Westerleigh|    28|
|Charleston/Totten...|   713|
|      Newark Airport|   739|
|          Douglaston|   860|
|East Concourse/Co...|  4112|
|      Pelham Parkway|  2041|
|          Mount Hope|  2859|
|         Marble Hill|   626|
|           Rego Park|  2590|
|Upper East Side S...|757472|
|       Dyker Heights|   743|
|Heartland Village...|   145|
|   Kew Gardens Hills|  1814|
|       Rikers Island|     6|
|             Bayside|  1257|
|     Jackson Heights|  5949|
+--------------------+------+
only showing top 20 rows



In [42]:
df = df.drop('dropoff_zone','pickup_zone')

## pickup_date, dropoff_date 칼럼 제거
- 주말인지 여부 파생변수

In [45]:
df.groupby('pickup_date').count().show()

+-----------+-----+
|pickup_date|count|
+-----------+-----+
| 2021-06-22|96228|
| 2021-01-27|51295|
| 2021-05-12|84766|
| 2021-07-30|96221|
| 2021-07-20|89403|
| 2021-04-29|80247|
| 2021-07-17|92299|
| 2021-07-23|96209|
| 2021-04-24|76788|
| 2009-01-01|  103|
| 2021-02-15|39291|
| 2021-03-22|58303|
| 2021-05-03|73349|
| 2021-07-19|86298|
| 2021-01-18|36135|
| 2021-01-25|46181|
| 2021-02-02|28437|
| 2021-07-10|91140|
| 2021-06-04|92975|
| 2021-04-25|48149|
+-----------+-----+
only showing top 20 rows



In [46]:
pickup_date_range = df.select(F.min("pickup_date").alias("min_pickup_date"),
                              F.max("pickup_date").alias("max_pickup_date")).show()

dropoff_date_range = df.select(F.min("dropoff_date").alias("min_dropoff_date"),
                               F.max("dropoff_date").alias("max_dropoff_date")).show()

+---------------+---------------+
|min_pickup_date|max_pickup_date|
+---------------+---------------+
|     2004-04-04|     2029-05-05|
+---------------+---------------+



+----------------+----------------+
|min_dropoff_date|max_dropoff_date|
+----------------+----------------+
|      2004-04-05|      2029-05-05|
+----------------+----------------+



In [47]:
# 'pickup_date'와 'dropoff_date'에서 요일 추출 (1=월요일, 7=일요일)
df = df.withColumn("pickup_day_of_week", F.dayofweek(df["pickup_date"]))
df = df.withColumn("dropoff_day_of_week", F.dayofweek(df["dropoff_date"]))

# 주말인지 아닌지를 나타내는 파생변수 생성 (주말이면 1, 아니면 0)
df = df.withColumn("is_pickup_weekend", F.when((df["pickup_day_of_week"] == 6) | (df["pickup_day_of_week"] == 7), 1).otherwise(0))
df = df.withColumn("is_dropoff_weekend", F.when((df["dropoff_day_of_week"] == 6) | (df["dropoff_day_of_week"] == 7), 1).otherwise(0))

# 이제 'pickup_day_of_week'와 'dropoff_day_of_week' 칼럼 삭제
df = df.drop("pickup_day_of_week", "dropoff_day_of_week")

In [49]:
df = df.drop('pickup_date','dropoff_date')

## 뷰 생성

In [19]:
df.createOrReplaceTempView('comb')

In [20]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_time>0
'''

spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2021-03-07|          1|
| 2021-03-13|          1|
| 2021-03-05|          1|
| 2021-03-06|          1|
| 2021-03-13|          1|
| 2021-03-11|          1|
| 2021-03-12|          1|
| 2021-03-20|          1|
| 2021-03-12|          1|
| 2021-03-06|          1|
| 2021-03-21|          1|
| 2021-03-24|          1|
| 2021-03-02|          1|
| 2021-03-17|          1|
| 2021-03-16|          1|
| 2021-03-03|          1|
| 2021-03-13|          1|
| 2021-03-07|          1|
| 2021-03-13|          1|
| 2021-03-18|          1|
+-----------+-----------+
only showing top 20 rows



In [21]:
query = '''
select pickup_date, pickup_time 
from comb 
where pickup_date<'2020-12-31'
'''
spark.sql(query).show()

+-----------+-----------+
|pickup_date|pickup_time|
+-----------+-----------+
| 2009-01-01|         21|
| 2009-01-01|         16|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          2|
| 2008-12-31|         13|
| 2009-01-01|          1|
| 2009-01-01|          0|
| 2009-01-01|          3|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          8|
| 2009-01-01|          2|
| 2008-12-31|         23|
| 2009-01-01|          0|
| 2009-01-01|          0|
| 2009-01-01|          0|
+-----------+-----------+
only showing top 20 rows



In [26]:
spark.sql(query).explain()

== Physical Plan ==
*(4) HashAggregate(keys=[dropoff_time#79, trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, total_amount#32], functions=[])
+- Exchange hashpartitioning(dropoff_time#79, trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, total_amount#32, 200), ENSURE_REQUIREMENTS, [id=#1900]
   +- *(3) HashAggregate(keys=[dropoff_time#79, knownfloatingpointnormalized(normalizenanandzero(trip_distance#20)) AS trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, knownfloatingpointnormalized(normalizenanandzero(tip_amount#29)) AS tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, knownfloatingpointnormalized(normalizenanandzero(total_amount#32)) AS total_amount#32], functions=[])
      +- *(3) Project

In [27]:
query2 = '''
select pickup_date, pickup_time 
from comb 
where pickup_time > 0 and pickup_time<=12
'''
spark.sql(query2).explain()

== Physical Plan ==
*(4) HashAggregate(keys=[dropoff_time#79, trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, total_amount#32], functions=[])
+- Exchange hashpartitioning(dropoff_time#79, trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, total_amount#32, 200), ENSURE_REQUIREMENTS, [id=#2007]
   +- *(3) HashAggregate(keys=[dropoff_time#79, knownfloatingpointnormalized(normalizenanandzero(trip_distance#20)) AS trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, knownfloatingpointnormalized(normalizenanandzero(tip_amount#29)) AS tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, knownfloatingpointnormalized(normalizenanandzero(total_amount#32)) AS total_amount#32], functions=[])
      +- *(3) Project

In [28]:
query3 = '''
select pickup_date , count(*) as trip_count
from comb 
where pickup_time > 0
group by pickup_date
order by pickup_date
'''
spark.sql(query3).explain()

== Physical Plan ==
*(6) Sort [pickup_date#76 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(pickup_date#76 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#2153]
   +- *(5) HashAggregate(keys=[pickup_date#76], functions=[count(1)])
      +- Exchange hashpartitioning(pickup_date#76, 200), ENSURE_REQUIREMENTS, [id=#2149]
         +- *(4) HashAggregate(keys=[pickup_date#76], functions=[partial_count(1)])
            +- *(4) HashAggregate(keys=[dropoff_time#79, trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, total_amount#32], functions=[])
               +- Exchange hashpartitioning(dropoff_time#79, trip_distance#20, pickup_date#76, dropoff_date#77, VendorID#16, pickup_time#78, tip_amount#29, pickup_zone#80, payment_type#25, passenger_count#19, dropoff_zone#81, total_amount#32, 200), ENSURE_REQUIREMENTS, [id=#2144]
                  +- *(3) HashAggregate(keys=[drop

## 운행 거리와 요금의 상관관계

In [22]:
query = '''
select corr(trip_distance, total_amount) as correlation
from comb
'''

spark.sql(query).show()

+--------------------+
|         correlation|
+--------------------+
|0.001136088625537...|
+--------------------+



## 피크 시간대 요금 분석

In [23]:
# 시간대별 승차건수, 요금 
query = '''
SELECT pickup_time, COUNT(*) AS counting, SUM(total_amount) AS total_fare
FROM comb
GROUP BY pickup_time
ORDER BY counting DESC
''' 

spark.sql(query).show()

+-----------+--------+--------------------+
|pickup_time|counting|          total_fare|
+-----------+--------+--------------------+
|         18| 1090676|2.1214896040000785E7|
|         15| 1057248|1.9345986820000723E7|
|         17| 1054550|2.0898812310000747E7|
|         14| 1019788| 1.817586856000068E7|
|         16| 1017955|1.9940757080000717E7|
|         13|  959324|1.7323265730000548E7|
|         19|  950564|1.8294262630000595E7|
|         12|  929874|1.6655357440000482E7|
|         11|  829942|1.4493488710000403E7|
|         10|  755816|1.3230574760000302E7|
|         20|  719753|1.3683735260000315E7|
|          9|  684362|1.2198736130000228E7|
|         21|  613558|1.1972544470000198E7|
|          8|  590053|1.0542565370000144E7|
|         22|  547719|1.1058150650000159E7|
|         23|  442166|    9448824.75000007|
|          7|  380850|   7153067.109999945|
|          0|  299802|   6790455.609999943|
|          6|  199243|  3988966.4499999736|
|          1|  174970|  3931215.

## 지불 유형별 요금, 팁 

In [24]:
query = '''
SELECT payment_type, SUM(total_amount) AS total_fare, SUM(tip_amount) AS tip
FROM comb
GROUP BY payment_type
ORDER BY total_fare DESC
''' 

spark.sql(query).show()

+------------+-------------------+--------------------+
|payment_type|         total_fare|                 tip|
+------------+-------------------+--------------------+
|           1|2.246290602700514E8|3.1532255180000182E7|
|           2|5.009360277998641E7|              277.31|
|           3| 1608725.3299999977|               50.42|
|           4|  345319.1099999997|                 6.1|
+------------+-------------------+--------------------+



## 승차지역/하차지역별 평균거리, 요금

In [25]:
query = '''
SELECT AVG(trip_distance) AS avg_distance,
        AVG(total_amount) AS avg_fare
FROM comb
GROUP BY pickup_zone, dropoff_zone
ORDER BY avg_distance DESC
''' 

spark.sql(query).show()

+------------------+------------------+
|      avg_distance|          avg_fare|
+------------------+------------------+
|         205266.12|             47.63|
|         167305.08|              76.0|
|         131123.07|             61.18|
|          77093.46|             19.04|
|         70968.425|             53.22|
|        65796.0925|32.072500000000005|
|          63336.38|50.620000000000005|
|          56564.13|             51.08|
|          54215.12|33.269999999999996|
|52683.263333333336| 49.25666666666666|
|49552.270000000004|            70.025|
|         45886.492|             51.65|
| 41189.96666666667|             59.79|
|        32914.5775|26.325000000000003|
|          31690.14|53.605000000000004|
|        31568.9575|           71.5875|
|29940.403333333335|40.803333333333335|
| 29903.07666666667|             53.49|
|29279.048749999998| 51.75625000000001|
|28037.878000000004| 40.12800000000001|
+------------------+------------------+
only showing top 20 rows



## 팁의 비율에 따른 거리, 여행 건수

In [26]:
# 팁 비율별 여행 건수와 총 거리와 거리 평균
query = '''
SELECT 
    tip_ratio,
    COUNT(*) AS trip_count,
    SUM(trip_distance) AS total_distance,
    AVG(trip_distance) AS avg_distance
FROM 
    comb
GROUP BY 
    tip_ratio
ORDER BY 
    tip_ratio
''' 

spark.sql(query).show()

+--------------------+----------+--------------------+------------------+
|           tip_ratio|trip_count|      total_distance|      avg_distance|
+--------------------+----------+--------------------+------------------+
|                 0.0|   4208688|5.4922750050000034E7|13.049850701691367|
|2.379196307487331E-5|         1|                 0.0|               0.0|
|3.024071610015725...|         1|                62.4|              62.4|
|3.251398101183509E-5|         1|               67.87|             67.87|
|3.329892444474043...|         1|                 0.0|               0.0|
|3.698772007693446E-5|         1|               58.23|             58.23|
|4.992261993909440...|         2|                 0.0|               0.0|
|5.003502451716201...|         1|               50.56|             50.56|
|5.076915266284206E-5|         1|                 0.0|               0.0|
|5.553395901593825E-5|         1|                 0.0|               0.0|
|5.569169079973268E-5|         1|     

## 총 요금 예측 모델링

In [50]:
df.printSchema()

root
 |-- VendorID: integer (nullable = false)
 |-- pickup_time: integer (nullable = true)
 |-- dropoff_time: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = false)
 |-- is_pickup_weekend: integer (nullable = false)
 |-- is_dropoff_weekend: integer (nullable = false)



In [54]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

train, test = df.randomSplit([0.8,0.2], seed=42)

num_features = ['trip_distance','tip_amount']
cat_features = ['VendorID', 'pickup_time','dropoff_time','passenger_count',
               'payment_type','is_pickup_weekend','is_dropoff_weekend']
target = 'total_amount'

# StringIndexer를 사용하여 범주형 변수들을 인덱스로 변환
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in cat_features]

# 원-핫 인코딩
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_onehot") for col in cat_features]

# 벡터화
assembler = VectorAssembler(inputCols=num_features + [col + "_onehot" for col in cat_features], outputCol="features")

# 선형 회귀 모델 정의
lr = LinearRegression(featuresCol="features", labelCol=target)

# 파이프라인 정의
pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])

# 모델 학습
model = pipeline.fit(train)

# 예측
predictions = model.transform(test)
predictions.select("prediction", target, "features").show(10)

# RMSE
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol=target, metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(rmse)

24/12/16 12:57:53 WARN Instrumentation: [5c06a51f] regParam is zero, which might cause numerical instability and overfitting.
24/12/16 13:01:00 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/16 13:01:00 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
24/12/16 13:01:00 WARN Instrumentation: [5c06a51f] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


+------------------+------------+--------------------+
|        prediction|total_amount|            features|
+------------------+------------+--------------------+
|27.062506913121258|        10.3|(62,[20,43,49,59]...|
| 11.81892595463748|        29.0|(62,[20,43,49,57,...|
| 11.81892595463748|       53.12|(62,[20,43,49,57,...|
|16.473135535283358|        8.15|(62,[0,1,20,43,49...|
|19.231205517917118|        9.45|(62,[0,1,20,43,49...|
|17.156575536809616|        9.95|(62,[0,1,20,43,49...|
|16.858230372080165|         8.3|(62,[0,20,43,49,5...|
|18.714149660383466|        10.8|(62,[0,1,20,43,49...|
|18.197069428269018|       11.15|(62,[0,1,20,43,49...|
|18.535683651868872|       12.35|(62,[0,1,20,43,49...|
+------------------+------------+--------------------+
only showing top 10 rows



231.9382230804608


In [55]:
spark.stop()